In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Height"
cohort = "GSE117525"

# Input paths
in_trait_dir = "../../input/GEO/Height"
in_cohort_dir = "../../input/GEO/Height/GSE117525"

# Output paths
out_data_file = "../../output/preprocess/Height/GSE117525.csv"
out_gene_data_file = "../../output/preprocess/Height/gene_data/GSE117525.csv"
out_clinical_data_file = "../../output/preprocess/Height/clinical_data/GSE117525.csv"
json_path = "../../output/preprocess/Height/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll provide a corrected implementation for this step.

```python
import pandas as pd
import os
import numpy as np
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# By examining the background information, this study focuses on skeletal muscle transcriptome
# which suggests it contains gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For height trait:
# Key 4 contains height information (e.g., 'height (m): 1.94')
trait_row = 4  

# For age:
# Key 3 contains age information (e.g., 'age (yrs): 21')
age_row = 3

# For gender:
# Key 1 contains gender information (e.g., 'Sex: M', 'Sex: F')
gender_row = 1

# 2.2 Data Type Conversion

# Height conversion function
def convert_trait(value):
    if pd.isna(value):
        return None
    try:
        # Extract the height value after the colon
        if "height (m):" in value:
            height_str = value.split("height (m):")[1].strip()
            return float(height_str)
        else:
            return None
    except:
        return None

# Age conversion function
def convert_age(value):
    if pd.isna(value):
        return None
    try:
        # Extract the age value after the colon
        age_str = value.split("age (yrs):")[1].strip()
        return float(age_str)
    except:
        return None

# Gender conversion function
def convert_gender(value):
    if pd.isna(value):
        return None
    # Convert 'Sex: F' to 0 and 'Sex: M' to 1
    if "Sex: F" in value:
        return 0
    elif "Sex: M" in value:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Save initial metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a sample characteristics DataFrame from the provided dictionary
    sample_chars = {
        0: ['tissue: vastus lateralis'], 
        1: ['Sex: M', 'Sex: F'], 
        2: ['subjectid: DSMT22', 'subjectid: 06OSN', 'subjectid: 4007', 'subjectid: 8003', 'subjectid: 8011', 'subjectid: 8027', 'subjectid: 8044', 'subjectid: 8082', 'subjectid: 8093', 'subjectid: 4058', 'subjectid: DSMT 23', 'subjectid: 11ETK', 'subjectid: 4010', 'subjectid: 4065', 'subjectid: DSMT 24', 'subjectid: 08ACN', 'subjectid: 4046', 'subjectid: 8004', 'subjectid: 8012', 'subjectid: 8028', 'subjectid: 8046', 'subjectid: 8095', 'subjectid: GUJ', 'subjectid: 4069', 'subjectid: DSMT 25', 'subjectid: 12AEY', 'subjectid: 8074', 'subjectid: 4074', 'subjectid: DSMT 28', 'subjectid: 02AET'], 
        3: ['age (yrs): 21', 'age (yrs): 22', 'age (yrs): 83', 'age (yrs): 77', 'age (yrs): 85', 'age (yrs): 79', 'age (yrs): 74', 'age (yrs): 72', 'age (yrs): 73', 'age (yrs): 93', 'age (yrs): 66', 'age (yrs): 18', 'age (yrs): 23', 'age (yrs): 87', 'age (yrs): 89', 'age (yrs): 81', 'age (yrs): 91', 'age (yrs): 84', 'age (yrs): 80', 'age (yrs): 90', 'age (yrs): 25', 'age (yrs): 96', 'age (yrs): 26', 'age (yrs): 19', 'age (yrs): 76', 'age (yrs): 78', 'age (yrs): 86', 'age (yrs): 68', 'age (yrs): 67', 'age (yrs): 75'], 
        4: ['height (m): 1.94', 'height (m): 1.84', 'height (m): 1.63', 'height (m): 1.76', 'height (m): 1.66', 'height (m): 1.56', 'height (m): 1.86', 'height (m): 1.59', 'height (m): 1.77', 'height (m): 1.90', 'height (m): 1.69', 'weight (kg): 119.00', 'weight (kg): 86.40', 'weight (kg): 81.00', 'weight (kg): 85.80', 'weight (kg): 72.60', 'height (m): 1.68', 'height (m): 1.97', 'height (m): 1.72', 'height (m): 1.74', 'height (m): 1.58', 'height (m): 1.55', 'height (m): 1.78', 'height (m): 1.73', 'weight (kg): 90.60', 'weight (kg): 56.80', 'weight (kg): 80.40', 'weight (kg): 52.40', 'weight (kg): 89.40', 'weight (kg): 69.90'], 
        5: ['weight (kg): 94.40', 'weight (kg): 68.20', 'weight (kg): 62.00', 'weight (kg): 115.60', 'weight (kg): 86.60', 'weight (kg): 79.80', 'weight (kg): 82.60', 'weight (kg): 83.60', 'weight (kg): 80.80', 'weight (kg): 78.20', 'weight (kg): 105.00', 'weight (kg): 82.20', 'weight (kg): 72.40', 'bmi (kg/m2): 38.40', 'bmi (kg/m2): 31.40', 'bmi (kg/m2): 33.10', 'bmi (kg/m2): 31.10', 'bmi (kg/m2): 23.50', 'bmi (kg/m2): 28.60', 'weight (kg): 69.60', 'weight (kg): 75.10', 'weight (kg): 67.20', 'weight (kg): 74.00', 'weight (kg): 87.60', 'weight (kg): 58.60', 'weight (kg): 77.40', 'weight (kg): 51.60', 'weight (kg): 88.80', 'weight (kg): 70.80', 'weight (kg): 95.00'],
        6: ['bmi (kg/m2): 25.08', 'bmi (kg/m2): 20.10', 'bmi (kg/m2): 23.34', 'bmi (kg/m2): 37.30',


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Let's first import the necessary libraries and examine the available data
import os
import json
import pandas as pd
import numpy as np
from typing import Callable, Optional, Dict, Any

# Let's check what files are available in the cohort directory
try:
    files = os.listdir(in_cohort_dir)
    print(f"Files in directory: {files}")
except Exception as e:
    print(f"Error accessing directory: {e}")
    files = []

# Check for gene expression data by looking for large matrix files
is_gene_available = False
for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    if file.endswith('.txt') or file.endswith('.csv'):
        try:
            # Check file size - gene expression files are typically large
            file_size = os.path.getsize(file_path) / (1024 * 1024)  # Size in MB
            if file_size > 1:  # If file is larger than 1MB, it might contain gene expression data
                is_gene_available = True
                print(f"Potential gene expression data found in: {file} (Size: {file_size:.2f} MB)")
                break
        except Exception as e:
            print(f"Error checking file {file}: {e}")

# Try to load sample characteristics
sample_characteristics = {}
for file in files:
    if 'characteristics' in file.lower() and file.endswith('.json'):
        try:
            with open(os.path.join(in_cohort_dir, file), 'r') as f:
                sample_characteristics = json.load(f)
                print(f"Loaded sample characteristics from: {file}")
                break
        except Exception as e:
            print(f"Error loading sample characteristics from {file}: {e}")

# Try to load background information
background_info = {}
for file in files:
    if 'background' in file.lower() and file.endswith('.json'):
        try:
            with open(os.path.join(in_cohort_dir, file), 'r') as f:
                background_info = json.load(f)
                print(f"Loaded background information from: {file}")
                break
        except Exception as e:
            print(f"Error loading background information from {file}: {e}")

# Analyze what we have
print("Background Information:")
print(background_info)
print("\nSample Characteristics:")
print(sample_characteristics)

# Determine if trait, age, and gender data are available
trait_row = None
age_row = None
gender_row = None

if sample_characteristics:
    print("\nUnique values for each key in sample_characteristics:")
    for key, values in sample_characteristics.items():
        unique_values = set(values)
        print(f"Key {key}: {unique_values}")
        
        # Look for height information in the values
        if any('height' in str(v).lower() for v in unique_values) or any('tall' in str(v).lower() for v in unique_values):
            trait_row = key
        
        # Look for age information
        if any('age' in str(v).lower() for v in unique_values) or any('years' in str(v).lower() for v in unique_values):
            age_row = key
        
        # Look for gender information
        if any('gender' in str(v).lower() for v in unique_values) or any('sex' in str(v).lower() for v in unique_values) or any('male' in str(v).lower() for v in unique_values) or any('female' in str(v).lower() for v in unique_values):
            gender_row = key
else:
    print("No sample characteristics data available to analyze.")

# Define conversion functions
def convert_trait(value):
    if value is None:
        return None
    
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    
    try:
        # Try to convert to float for height (assuming in cm or inches)
        return float(value)
    except:
        # If conversion fails, try to extract numeric values
        import re
        numeric_value = re.search(r'(\d+\.?\d*)', str(value))
        if numeric_value:
            return float(numeric_value.group(1))
        return None

def convert_age(value):
    if value is None:
        return None
    
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    
    try:
        # Try to convert to float for age (assuming in years)
        return float(value)
    except:
        # If conversion fails, try to extract numeric values
        import re
        numeric_value = re.search(r'(\d+\.?\d*)', str(value))
        if numeric_value:
            return float(numeric_value.group(1))
        return None

def convert_gender(value):
    if value is None:
        return None
    
    if ':' in str(value):
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = str(value).lower()
    
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    else:
        return None

# Check if trait data is available
is_trait_available = trait_row is not None
print(f"\nData availability assessment:")
print(f"Gene expression data available: {is_gene_available}")
print(f"Trait (Height) data available: {is_trait_available}")
print(f"Age data available: {age_row is not None}")
print(f"Gender data available: {gender_row is not None}")

# Validate and save cohort information
initial_validation = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait data is available
if is_trait_available:
    try:
        # Create a dataframe from sample_characteristics
        clinical_df = pd.DataFrame(sample_characteristics)
        
        # Use the geo_select_clinical_features function to extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_df,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age if age_row is not None else None,
            gender_row=gender_row,
            convert_gender=convert_gender if gender_row is not None else None
        )
        
        # Preview the dataframe
        print("\nClinical Features Preview:")
        print(preview_df(selected_clinical_df))
        
        # Save the clinical data to a CSV file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to: {out_clinical_data_file}")
    except Exception as e:
        print(f"Error processing clinical data: {e}")
else:
    print("Skipping clinical feature extraction as trait data is not available.")


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


### Step 5: Gene Identifier Review

In [ ]:
# Analyzing the gene identifiers in the gene expression data

# Examples from the data:
# - "100009676_at"
# - "10000_at"
# - "10001_at"

# These appear to be probe IDs from a microarray platform, not standard human gene symbols
# Standard human gene symbols would be like BRCA1, TP53, etc.
# The "_at" suffix is characteristic of Affymetrix microarray probe IDs
# These identifiers will need to be mapped to standard gene symbols for analysis

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


### Step 7: Gene Identifier Mapping

In [ ]:
# Extract gene annotation data from the SOFT file using custom approach
# Let's extract the gene annotation data using get_gene_annotation
gene_annotation = get_gene_annotation(soft_file)

# Identify the columns we need for mapping
# Looking at the gene expression data, we have IDs like "100009676_at"
# In the gene annotation, we see "ID" column has similar identifiers (e.g., "1_at")
# The "Description" column appears to have gene names/descriptions

# Get the mapping between probe IDs and gene descriptions
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'Description')

# Print mapping information for verification
print(f"Found {len(mapping_df)} probe-to-gene mappings")
print(f"Sample mappings (first 5 rows):")
print(mapping_df.head())

# Now apply the gene mapping to convert probe measurements to gene expression
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print information about the resulting gene expression data
print(f"\nAfter mapping: gene expression data has {gene_data.shape[0]} genes and {gene_data.shape[1]} samples")
print("First 5 genes:")
print(gene_data.head())

# Save the gene expression data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


### Step 8: Data Normalization and Linking

In [ ]:
# Get file paths first
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 1. Let's correctly read the clinical data first to extract height information
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Inspect the clinical data to find height information
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# Based on the sample_characteristics_dict from Step 1, height data is in row 4
# Let's extract height and other clinical information
trait_row = 4  # Height information
age_row = 3    # Age information 
gender_row = 1 # Gender information (Sex: M/F)

# Define conversion functions
def convert_trait(value):
    if pd.isna(value):
        return None
    try:
        # Extract the height value after the colon
        if "height (m):" in value:
            height_str = value.split("height (m):")[1].strip()
            return float(height_str)
        else:
            return None
    except:
        return None

def convert_age(value):
    if pd.isna(value):
        return None
    try:
        # Extract the age value after the colon
        age_str = value.split("age (yrs):")[1].strip()
        return float(age_str)
    except:
        return None

def convert_gender(value):
    if pd.isna(value):
        return None
    # Convert 'Sex: F' to 0 and 'Sex: M' to 1
    if "Sex: F" in value:
        return 0
    elif "Sex: M" in value:
        return 1
    else:
        return None

# Extract clinical features including height
clinical_features = geo_select_clinical_features(
    clinical_data, 
    trait=trait, 
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_features.shape}")
print(f"Clinical data preview:")
print(clinical_features.head())

# Now extract gene expression data
# 1. Extract gene expression data using the get_genetic_data function
gene_data = get_genetic_data(matrix_file)
print(f"Gene data shape before normalization: {gene_data.shape}")

# 2. Normalize gene symbols using the NCBI Gene database
# If normalization results in empty data, use the original data as a fallback
try:
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Check if we lost all genes during normalization and use fallback if needed
    if normalized_gene_data.empty:
        print("Warning: All genes were filtered out during normalization. Using fallback approach.")
        # Fallback: Use original data with the 'at' suffixes stripped
        gene_data.index = gene_data.index.str.split('_').str[0]
        normalized_gene_data = gene_data.groupby(gene_data.index).mean()
        print(f"Fallback gene data shape: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Gene data saved to {out_gene_data_file}")
    
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False
    normalized_gene_data = gene_data  # Use original data if normalization fails

# 3. Link clinical and genetic data - transform gene data to have samples as rows
if is_gene_available and normalized_gene_data is not None:
    # Transpose normalized_gene_data to have samples as rows
    gene_data_t = normalized_gene_data.T
    
    # Create linked data - we need to ensure sample IDs align
    # Note: We link based on sample IDs, which should be in the columns of normalized_gene_data
    # and the index of clinical_features.T
    linked_data = pd.concat([clinical_features.T, gene_data_t], axis=1)
    print(f"Linked data shape: {linked_data.shape}")
    
    # Check for the completeness of the trait data
    trait_available = linked_data[trait].notna().sum() > 0
    print(f"Number of samples with {trait} data: {linked_data[trait].notna().sum()}")
    
    # Handle missing values
    if trait_available:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"After handling missing values, linked data shape: {linked_data.shape}")
        
        # Check for trait bias
        is_trait_available = True
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        
        note = "Dataset contains both gene expression and height data."
    else:
        is_trait_available = False
        is_biased = False
        note = "Dataset does not contain sufficient height measurements for analysis."
else:
    # Create a minimal dataframe with just the trait column
    linked_data = clinical_features.T
    is_trait_available = linked_data[trait].notna().sum() > 0
    is_biased = False
    is_gene_available = False
    note = "Dataset does not contain usable gene expression data."

# 4 & 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save linked data if usable
print(f"Dataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset is not suitable for {trait} association studies.")